# Toronto Neighbourhoods

In [29]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import folium # map rendering library
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
content = result.content
soup = BeautifulSoup(content,'lxml')
table = soup.find_all('table')[0]

## Clean Up Postal Codes / Borough and Neighbourhood data

In [3]:
postal_codes = pd.DataFrame()

for tr in table.find_all('tr')[1:]:
    td = tr.find_all("td")

    postal_code = str(td[0].contents[0])

    borough = str(td[1].contents[0])
    start = borough.find('">')+2
    start
    end = borough.find('</a>')
    end
    borough = borough[start:end]
    # Replace 'or' with York
    if (borough == 'or'):
        borough = 'York'
    if (borough == 'ississaug'):
        borough = 'Mississauga'
        
    neighbourhood = str(td[2].contents[0])
    if (neighbourhood[0] == "<"):
        start = neighbourhood.find('">')+2
        start
        end = neighbourhood.find('</a>')
        end
        neighbourhood = neighbourhood[start:end]    
    if (neighbourhood[-1] == "\n"):
        start = 0
        end = neighbourhood.find('\n')
        neighbourhood = neighbourhood[start:end]
    
    row_df = pd.DataFrame([[postal_code, borough, neighbourhood]])
    postal_codes = pd.concat([postal_codes, row_df],ignore_index=True)

# Set column Names    
postal_codes.columns=['PostalCode','Borough','Neighbourhood']

# Remove 'Not assigned' boroughs
postal_codes = postal_codes[postal_codes['Borough']!='ot assigne']

# Re-index
postal_codes = postal_codes.reset_index(drop=True)

# If a cell has a borough but a Not assigned neighbourhood, then the neighbourhood will be the same as the borough
postal_codes.loc[postal_codes['Neighbourhood'] == 'Not assigned','Neighbourhood'] = postal_codes['Borough']

postal_codes.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Combine neighbourhoods from the same borough

In [4]:
#Combine neighbourhoods from the same borough
postal_codes_combined = postal_codes.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
postal_codes = postal_codes[['PostalCode','Borough','Neighbourhood']]

postal_codes_combined=postal_codes_combined.reset_index()
postal_codes_combined = postal_codes_combined[['PostalCode','Borough','Neighbourhood']]

postal_codes_combined.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
postal_codes_combined.shape

(103, 3)

## Associate postal codes with coordinates

In [6]:
# Coordinates
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates

toronto_neighbourhoods = pd.concat([postal_codes_combined,coordinates],axis=1)
toronto_neighbourhoods = toronto_neighbourhoods[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
toronto_neighbourhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a map of Toronto

In [7]:
address = 'Toronto, ON'

latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_neighbourhoods['Latitude'], toronto_neighbourhoods['Longitude'], toronto_neighbourhoods['Borough'], toronto_neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(toronto_neighbourhoods, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## FourSquare Integration

In [8]:
CLIENT_ID = 'XT3PLM1C0BXYHLRKF20DAUEINAF0Z0HPY2SECZSNATD4BSOJ' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XT3PLM1C0BXYHLRKF20DAUEINAF0Z0HPY2SECZSNATD4BSOJ
CLIENT_SECRET:JQVVUKAHE12LS0D4DVHBEVPKYUWXHSYMPPF1R3P55JT54QFV


In [9]:
toronto_neighbourhoods.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [10]:
neighbourhood_latitude = toronto_neighbourhoods.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_neighbourhoods.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_neighbourhoods.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [11]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XT3PLM1C0BXYHLRKF20DAUEINAF0Z0HPY2SECZSNATD4BSOJ&client_secret=JQVVUKAHE12LS0D4DVHBEVPKYUWXHSYMPPF1R3P55JT54QFV&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d63552a9ba3e5002c7170ae'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [15]:
## Explore Neighbourhoods

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_neighbourhoods['Neighbourhood'],
                                   latitudes=toronto_neighbourhoods['Latitude'],
                                   longitudes=toronto_neighbourhoods['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [18]:
## Analyze Neighbourhoods


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.01,0.000000
1,Agincourt,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.0,0.00,0.000000
6,Bayview Village,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.043478,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
8,Berczy Park,0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000


In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,American Restaurant,Breakfast Spot,Restaurant,Asian Restaurant
1,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Chinese Restaurant,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Liquor Store,Sandwich Place,Fried Chicken Joint,Coffee Shop,Beer Store,Pizza Place,Fast Food Restaurant,Empanada Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Skating Rink,Coffee Shop,Athletics & Sports,Pub,Dog Run,Diner


## Cluster Neighbourhoods

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,History Museum,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Pizza Place,Breakfast Spot,Medical Center,Electronics Store,Rental Car Location,Intersection,Donut Shop,Dive Bar,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Bakery,Caribbean Restaurant,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Drugstore,Doner Restaurant


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,History Museum,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,Scarborough,0.0,Mexican Restaurant,Pizza Place,Breakfast Spot,Medical Center,Electronics Store,Rental Car Location,Intersection,Donut Shop,Dive Bar,Dog Run
3,Scarborough,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Scarborough,0.0,Hakka Restaurant,Bakery,Caribbean Restaurant,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Drugstore,Doner Restaurant
5,Scarborough,0.0,Health & Beauty Service,Playground,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
6,Scarborough,0.0,Hobby Shop,Convenience Store,Department Store,Coffee Shop,Chinese Restaurant,Discount Store,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant
7,Scarborough,0.0,Bus Line,Bakery,Intersection,Bus Station,Fast Food Restaurant,Soccer Field,Park,Event Space,Ethiopian Restaurant,Empanada Restaurant
8,Scarborough,0.0,American Restaurant,Motel,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Discount Store
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Diner
10,Scarborough,0.0,Indian Restaurant,Gaming Cafe,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Donut Shop,Discount Store,Dive Bar,Dog Run


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Playground,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
30,North York,1.0,Airport,Park,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
40,East York,1.0,Park,Pizza Place,Convenience Store,Coffee Shop,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,1.0,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
50,Downtown Toronto,1.0,Park,Building,Playground,Trail,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
74,York,1.0,Park,Women's Store,Market,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Concert Hall,Farmers Market,Falafel Restaurant,Event Space
79,North York,1.0,Construction & Landscaping,Park,Bakery,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
90,Etobicoke,1.0,River,Park,Yoga Studio,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
98,York,1.0,Park,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,2.0,Cafeteria,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Comfort Food Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4.0,Garden,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Field
